## 匯入套件

In [1]:
import json
import nltk
import codecs
import string
import re
import os 
import pandas as pd
import numpy as np
import emot

In [2]:
from zhon.hanzi import punctuation
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora
from gensim import models
from gensim.models import word2vec
from pandas import json_normalize
from ckiptagger import WS, POS, NER
from ckiptagger import construct_dictionary
from warnings import simplefilter

In [3]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

## 資料夾內容，存成list

In [4]:
simplefilter(action='ignore', category=FutureWarning)

## 讀取所有json檔案，存成csv

In [5]:
##資料夾路徑
path='..\..\data統整\PTT\PTT_JASON_檔案\標記答案_最終答案'
json_data_list=[]

#遍歷路徑中資料夾，存成list
for file in os.listdir(path):
    
    #判斷子檔案的後綴字詞是否為.json
    if file.endswith('.json'):
        #將path與file找到檔案字串進行串接
        json_data=os.path.join(path,file)
        #讀取json檔
        load_json=json.load(codecs.open(json_data, 'r', 'utf-8-sig'))
        #萃取key為推文的部分，存成json_tweets
        json_tweets= load_json['推文']
        context_list=[]
        answer_list=[]
        new_dict={}
        
        #迴圈依序挑出字典的value
        for i in range(1, len(json_tweets)+1):
            
            #分別萃取出字典1到最後的value，在挑出下層字典的留言內容的value與答案的value
            context_list.append(json_tweets[str(i)]['留言內容'])
            answer_list.append(json_tweets[str(i)]['答案'])
            #以字典形式儲存
            new_dict={'留言內容':context_list,'答案':answer_list}
            
        
        #df為字典構成的dataframe   
        df = pd.DataFrame.from_dict(new_dict)
        #將dataframe新增到list(all_data_list)
        json_data_list.append(df)

## Json檔串接完保留成CSV檔

In [6]:
#將list以concat的方式進行串接
all_json_data=pd.concat(json_data_list)
#存成csv檔
all_json_data['答案'] = all_json_data['答案'].replace(0,'0').replace(1,'1')
all_json_data.to_csv (r"..\..\data統整\PTT\context.csv", index = False, header=True,encoding='utf_8_sig')

## 刪除URL

In [7]:
RemoveUrl=[]
for word,answer in zip(all_json_data['留言內容'],all_json_data['答案']):
    if ".html" not in word:
        word=re.sub(r"http\S+","", str(word))
        word=re.sub(r"goo.gl\S+","", str(word))
        RemoveUrl.append([word,answer])

## 載入停用詞表

In [8]:
stopword_data =pd.read_csv(r"..\..\字典\停用詞\stopword-extended.csv",encoding='utf_8_sig')

## 清除空格，拆分文本和內容

In [9]:
word_list=[]
answer_list=[]
for word,answer in RemoveUrl:
    word= word.replace(' ', '')
    word_list.append(word.lower())
    answer_list.append(answer)

## 情感字典

#### 匯入情感字典

In [10]:
S_d  = pd.read_csv(r"..\..\字典\情感字典\ChineseEmoBank\CVAW_SD\CVAW_all_SD.csv",sep="\t")

#### 新增一個空的CKIP字典，將情感詞導入

In [11]:
dict_for_CKIP={}
dict_for_CKIP = dict((el,1) for el in S_d['Word'])

In [12]:
emot_dict={}
emot_obj = emot.core.emot() 

for sentence in RemoveUrl:
    #假設表情字串存在表情符號
    emoji_temp=emot_obj.emoji(sentence)
    if len(emoji_temp['value'])!=0:
        for word in emoji_temp['value']:
            emot_dict[word]=1

In [13]:
dict_for_CKIP.update(emot_dict)

## word2Vec霸凌詞 

#### 載入word2Vec霸凌詞字典

In [14]:
word2Vec=pd.read_csv(r"..\..\字典\霸凌字典\word2Vec.csv",encoding='utf_8_sig')

#### word2Vec霸凌詞字典整理

In [15]:
word2Vec_word=[]
for word in word2Vec['all_20']:
    word2Vec_word.extend(word.lower().split())

In [16]:
word2Vec_dict = dict((el,1) for el in word2Vec_word)

#### 更新CKIP字典(word2Vec)

In [17]:
dict_for_CKIP.update(word2Vec_dict)

## 卡方霸凌詞

#### 載入卡方霸凌詞字典

In [18]:
chisquare=pd.read_csv(r"..\..\字典\霸凌字典\chisquare_news100.csv",encoding='utf_8_sig')

#### 卡方霸凌詞字典整理

In [19]:
chisquare_list=[]
for word in chisquare['字']:
    chisquare_list.extend(word.lower().split())

In [20]:
dict_chisquare = dict((el,1) for el in chisquare_list)

#### 更新要匯入CKIP字典的字典

In [21]:
dict_for_CKIP.update(dict_chisquare)

In [22]:
dict_for_CKIP = construct_dictionary(dict_for_CKIP)

## CKIP斷詞、詞性分析、專有名詞辨識

In [23]:
ws = WS(r"..\data")
pos = POS("../data")
ner = NER("../data")

C:\Users\Yang\anaconda3\envs\python37\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\Users\Yang\anaconda3\envs\python37\lib\site-packages\ckiptagger\model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\Users\Yang\anaconda3\envs\python37\lib\site-packages\ckiptagger\model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.r

In [24]:
ws_result_temp = ws(word_list,coerce_dictionary = dict_for_CKIP)

In [25]:
ws_results=[]
for sentence in  ws_result_temp:
    ws_temp=[]
    for word in sentence:
        word= word.replace(' ', '')
        if len(word) !=0:
            ws_temp.append(word)
    ws_results.append(ws_temp)

## 詞性與命名實體標註

In [26]:
pos_results = pos(ws_result_temp)

## 刪除停用詞

In [27]:
temp=[]
re_punctuation=[]
for sentence in ws_results:
    ws_temp=[]
    rp_temp=[]
    for word in sentence:
        word= word.translate(str.maketrans('', '', punctuation))
        word= word.translate(str.maketrans('', '', string.punctuation))
        pattern = re.compile(u'[^0-9a-zA-Zａ-ｚＡ-Ｚ\u4e00-\u9fa5\u3100-\u312F\u3040-\u309F\u30A0-\u30FF\u31F0-\u31FF.，,。？""]+')
        if len(word)!=0 and word not in stopword_data['停用詞'].tolist():
            ws_temp.append(word)
        if len(word)!=0:
            rp_temp.append(word)
    temp.append(ws_temp)
    re_punctuation.append(rp_temp)

#### 清除刪完停用詞後的空的評論
remove_stw_ws：刪完停用詞文本，remove_stw_ans：文本標註，lenContent：評論長度，ws_data：不刪停用詞文本，pos_data：不刪停用詞詞性

In [28]:
remove_stw_ws=[]
ws_data=[]
pos_data=[]
remove_stw_ans=[]
remove_stw_count=[]

for rsw_word,word,pos,answer,rp_word in zip(temp,re_punctuation,pos_results,answer_list,re_punctuation):
    if len(rsw_word) !=0:
        remove_stw_ws.append(rsw_word)
        ws_data.append(word)
        pos_data.append(pos)
        remove_stw_ans.append(answer)
        remove_stw_count.append(len(rp_word))

## 詞袋模型

#### 建立字典

In [29]:
dictionary = corpora.Dictionary(remove_stw_ws)

以迴圈遍歷斷詞後的list，取出所有分詞  
將所有分詞結果放入 dictionary.doc2bow  
dictionary.doc2bow(word) 裡的 word 是每一句評論的斷詞結果  
dictionary.doc2bow(word)方法將斷詞結果以詞袋模型展現  
將結果放入list當中  

In [30]:
words_bag = [dictionary.doc2bow(word) for word in remove_stw_ws]

In [31]:
id_words = [[(dictionary[id], count) for id, count in line] for line in words_bag]

#### 取出所有詞彙整成list，並刪除重複值

In [32]:
ws_list = list(set(np.concatenate(remove_stw_ws)))

## 建立空的Data Frame

In [33]:
zero_DF = np.zeros((len(id_words),len(ws_list)))
BOW_DF = pd.DataFrame(data = zero_DF, columns=ws_list)

## 將單詞次數寫入Data Frame

In [34]:
for index,id_word in enumerate(id_words):
    for word,count in id_word:
        BOW_DF.loc[index, word] =count

#### 將答案的list插入至Bag of words向量，存成csv檔案

In [35]:
BOW_DF=BOW_DF.assign(answer=remove_stw_ans)
BOW_DF.to_csv (r"..\..\Feature_Array\PTT\Main_Features\Bag_of_Word.csv", header=True,encoding='utf_8_sig',index=None)

In [36]:
BOW_DF['answer'].value_counts()[0]/4226

0.5016564126833886

In [37]:
BOW_DF['answer'].value_counts()[1]/4226

0.49834358731661144

## TF-IDF

#### 做出一個以空格斷詞的 list 方便跑TF-IDF

In [38]:
join_ws=[]
for word in remove_stw_ws:
    sentence=' '.join(word)
    join_ws.append(sentence)

In [39]:
def TF_IDF(max_feature):
    vectoriser = TfidfVectorizer(norm = None,token_pattern=r"(?u)\b\w+\b",max_features=max_feature)
    tf_idf_scores = vectoriser.fit_transform(join_ws)
    feature_names = vectoriser.get_feature_names()
    df_tf_idf = pd.DataFrame(tf_idf_scores.todense(), columns =  feature_names)
    return df_tf_idf

In [40]:
df_tf_idf_1000=TF_IDF(1000)
df_tf_idf_500=TF_IDF(500)

#### 將答案的list插入至TF-IDF向量，存成csv檔案

In [41]:
df_tf_idf_1000=df_tf_idf_1000.assign(answer=remove_stw_ans)
df_tf_idf_1000.to_csv (r"..\..\Feature_Array\PTT\Main_Features\TF-IDF_1000字.csv", header=True,encoding='utf_8_sig',index=None)

In [42]:
df_tf_idf_500=df_tf_idf_500.assign(answer=remove_stw_ans)
df_tf_idf_500.to_csv (r"..\..\Feature_Array\PTT\Main_Features\TF-IDF_500字.csv", header=True,encoding='utf_8_sig',index=None)

## 計算情感分數

#### 製作情感字典(key=詞:value=情感分數)

In [43]:
Sentiment_dict =  dict((word,value) for word,value in zip(S_d ['Word'], S_d ['Valence_Mean']))

#### 包含 文本、正向情感字數、負向情感字數、情感分數

In [44]:
Sentiment_list=[]
for sentence,count in zip(ws_data,remove_stw_count):
    p=0
    n=0
    sentiment_score=0
    for word in sentence:
        s=' '.join(sentence) 
        #在情感字典裡且情感分數>5為正向情感字
        if word in Sentiment_dict.keys() and Sentiment_dict[word]>5:
            p=(p+1)
         #在情感字典裡且情感分數<5為負向情感字
        if word in Sentiment_dict.keys() and Sentiment_dict[word]<5:
            n=(n+1)
        if word in Sentiment_dict.keys():
            sentiment_score=sentiment_score+(((Sentiment_dict[word]-5)/4)+1)
    #計算正向情感字比率 正向字字數/總字數        
    p=p/count
    #計算負向情感字比率 負向字字數/總字數        
    n=n/count
    
    Sentiment={'文本':s,'正向情感比率':p,'負向情感比率':n,'情感分數':sentiment_score}
    df=pd.DataFrame([Sentiment])
    Sentiment_list.append(df)
Sentiment_data=pd.concat(Sentiment_list)  

In [45]:
Sentiment_data.to_csv (r"..\..\Feature_Array\PTT\Extra_Features\Sentiment_data.csv", header=True,encoding='utf_8_sig',index=None)

## word2Vec霸凌詞分析

In [46]:
bully_list=[]
for sentence,count in zip(ws_data,remove_stw_count):
    bully_count=0
    for word in sentence:
        s=' '.join(sentence) 
        #假如霸凌詞有在list裡面
        if str(word) in word2Vec_word:
            bully_count+=1
    bully_count=bully_count/count
    bully_dict={'文本':s,'霸凌詞比率':bully_count}
    df=pd.DataFrame([bully_dict])
    bully_list.append(df)
    
bully_data=pd.concat(bully_list)

In [47]:
bully_data.to_csv (r"..\..\Feature_Array\PTT\Extra_Features\Word2Vec_Bully_data.csv", header=True,encoding='utf_8_sig',index=None)

## 卡方霸凌詞分析

In [48]:
chisquare_list=[]
for sentence,count in zip(ws_data,remove_stw_count):
    bully_count=0
    for word in sentence:
        s=' '.join(sentence) 
        #假如霸凌詞有在list裡面
        if str(word) in chisquare['字'].tolist():
            bully_count+=1
    bully_count=bully_count/count
    bully_dict={'文本':s,'霸凌詞比率':bully_count}
    df=pd.DataFrame([bully_dict])
    chisquare_list.append(df)

chisquare_data=pd.concat(chisquare_list)

In [49]:
chisquare_data.to_csv (r"..\..\Feature_Array\PTT\Extra_Features\Chisquare_Bully_data.csv", header=True,encoding='utf_8_sig',index=None)

## 代詞

In [50]:
pronoun=pd.read_csv(r"..\..\字典\詞性字典\人稱代詞.csv",encoding='utf_8_sig')

In [51]:
pronoun_list=[]
for sentence,count in zip(ws_data,remove_stw_count):
    first=0
    second=0
    third=0
    for word in sentence:
        s=' '.join(sentence)
        if word in pronoun['第一人稱'].tolist():
            first+=1
        if word in pronoun['第二人稱'].tolist():
            second+=1
        if word in pronoun['第三人稱'].tolist():
            third+=1

    first=first/count     
    second=second/count
    third=third/count

    
    pronoun_dict={'文本':s,'第一人稱比率':first,'第二人稱比率':second,'第三人稱比率':third}
    df=pd.DataFrame([pronoun_dict])
    pronoun_list.append(df)
pronoun_data=pd.concat(pronoun_list)  


In [52]:
pronoun_data.to_csv (r"..\..\Feature_Array\PTT\Extra_Features\Pronoun_data.csv", header=True,encoding='utf_8_sig',index=None)

### 統計特徵(留言長度+人稱代詞)

In [53]:
Statistical_Features=pronoun_data.assign(留言長度= remove_stw_count)

In [54]:
Statistical_Features.to_csv (r"..\..\Feature_Array\PTT\Extra_Features\Statistical_Features.csv", header=True,encoding='utf_8_sig',index=None)

## 詞性統計(48類)

#### 創建一個{key=所有詞性:value=0}的字典

In [55]:
pos_list= list(set(np.concatenate(pos_data)))
pos_list.sort()

pos_dict={}

for i in pos_list:
    pos_dict[i]=0

#### 創建一個空的詞性DataFrame

In [56]:
pos_zero_df = np.zeros((len(pos_data),len(pos_list)))
pos_df = pd.DataFrame(data = pos_zero_df, columns=pos_list)

#### 遍歷文本，將統計數值填入DataFrame當中

In [57]:
for index,list_pos in  enumerate(pos_data):
    for pos in list_pos:
        if pos in pos_dict.keys():
            pos_dict[pos]+=1
            pos_df.loc[index, pos]=pos_dict[pos]/len(list_pos)
    for i in pos_dict:
        pos_dict[i]=0            

#### 存成csv檔

In [58]:
pos_df.to_csv (r"..\..\Feature_Array\PTT\Extra_Features\POS_data.csv", header=True,encoding='utf_8_sig',index=None)

## 詞性統計(11類)

#### 匯入分類的詞性表

#### 創建一個{key=分類後詞性項目:value=0}的字典

In [59]:
pos_table=pd.read_csv(r"..\..\字典\詞性字典\詞性表.csv",encoding='utf_8_sig')

In [60]:
simple_pos_dict={}

for i in pos_table.columns:
    simple_pos_dict[i]=0

#### 創建一個空的10類詞性DataFrame

In [61]:
pos_simple_data = np.zeros((len(pos_data),len(simple_pos_dict)))
pos_simple_df = pd.DataFrame(data = pos_simple_data, columns=simple_pos_dict)

#### 遍歷文本，將統計數值填入DataFrame當中

In [62]:
for index,pos_sentence in enumerate(pos_data):
    for pos in pos_sentence:
        if pos in pos_table['A'].tolist():
            simple_pos_dict['A']+=1
            pos_simple_df.loc[index, 'A'] =simple_pos_dict['A']/len(pos_sentence)
            
        if pos in pos_table['C'].tolist():
            simple_pos_dict['C']+=1
            pos_simple_df.loc[index, 'C'] =simple_pos_dict['C']/len(pos_sentence)
            
        if pos in pos_table['D'].tolist():
            simple_pos_dict['D']+=1
            pos_simple_df.loc[index, 'D'] =simple_pos_dict['D']/len(pos_sentence)
            
        if pos in pos_table['FW'].tolist():
            simple_pos_dict['FW']+=1
            pos_simple_df.loc[index, 'FW'] =simple_pos_dict['FW']/len(pos_sentence)
            
        if pos in pos_table['I'].tolist():
            simple_pos_dict['I']+=1
            pos_simple_df.loc[index, 'I'] =simple_pos_dict['I']/len(pos_sentence)
            
        if pos in pos_table['N'].tolist():
            simple_pos_dict['N']+=1
            pos_simple_df.loc[index, 'N'] =simple_pos_dict['N']/len(pos_sentence)
            
        if pos in pos_table['P'].tolist():
            simple_pos_dict['P']+=1
            pos_simple_df.loc[index, 'P'] =simple_pos_dict['P']/len(pos_sentence)
            
        if pos in pos_table['SHI'].tolist():
            simple_pos_dict['SHI']+=1
            pos_simple_df.loc[index, 'SHI'] =simple_pos_dict['SHI']/len(pos_sentence)
            
        if pos in pos_table['T'].tolist():
            simple_pos_dict['T']+=1
            pos_simple_df.loc[index, 'T'] =simple_pos_dict['T']/len(pos_sentence)
            
        if pos in pos_table['V'].tolist():
            simple_pos_dict['V']+=1
            pos_simple_df.loc[index, 'V'] =simple_pos_dict['V']/len(pos_sentence)
        
        if pos in pos_table['Punctuation'].tolist():
            simple_pos_dict['Punctuation']+=1
            pos_simple_df.loc[index, 'Punctuation'] =simple_pos_dict['Punctuation']/len(pos_sentence)
        
    for i in simple_pos_dict.keys():
        simple_pos_dict[i]=0

#### 存成csv檔

In [63]:
pos_simple_df.to_csv (r"..\..\Feature_Array\PTT\Extra_Features\Simple_POS_data.csv", header=True,encoding='utf_8_sig',index=None)

## 表情符號

### 方法一、表情符號情感字典

#### 導入情感字典

In [64]:
emoji_dict_data=pd.read_csv(r"..\..\字典\表情符號字典\ijstable.csv",encoding='utf_8_sig')

#### 文本清理

In [65]:
emoji_dict_data=emoji_dict_data.dropna(subset=['Char'])

#### 建立表情符號字典

In [66]:
emoji_dictionary={}

for word,score in zip(emoji_dict_data['Char'],emoji_dict_data['Sentiment score']):
    emoji_dictionary[word]=float(score)

#### 計算評論表情符號比率&情感分數

In [67]:
emot_obj = emot.core.emot() 
emoji_list=[]
emot_list=[]

for sentence,count in zip(ws_data,remove_stw_count):
    
    emoji_count=0
    emoji_setiment_value=0.0
    
    emoji_temp=emot_obj.emoji(sentence)
    emot_list.append(emoji_temp)
    
    if len(emoji_temp['value'])!=0:
        for word in emoji_temp['value']:
            if  word in emoji_dictionary.keys():
                emoji_count+=1
                emoji_setiment_value+=emoji_dictionary[word]+1
                
    emoji_count=emoji_count/count
    
    emoji_temp_dict={'表情符號比率':emoji_count,'表情符號情感分數':emoji_setiment_value}
    df=pd.DataFrame([emoji_temp_dict])
    emoji_list.append(df)
emoji_data=pd.concat(emoji_list)  

In [68]:
emoji_data.to_csv (r"..\..\Feature_Array\PTT\Extra_Features\Dict_Emoji_data.csv", header=True,encoding='utf_8_sig',index=None)

#### 情感特徵與表情符號合併

In [69]:
All_Sentiment=pd.concat([Sentiment_data.iloc[:, :],emoji_data], axis=1)

In [70]:
All_Sentiment.to_csv (r"..\..\Feature_Array\PTT\Extra_Features\All_Sentiment.csv", header=True,encoding='utf_8_sig',index=None)

### 方法2、Sentiwordnet

#### 建立表情符號字典

In [71]:
emoji_dict={}
emot_obj = emot.core.emot() 

for word in emot_list:
    #假設表情字串存在表情符號
    if len(word['value'])!=0:
        #將其value 和 mean 導入自訂的字典當中，key:value，value：mean
        for item_word,item_mean in zip(word['value'],word['mean']):
            emoji_dict[item_word]=item_mean

#### 以正則表示式清除字典value的冒號

In [72]:
list_emoji_word=[]
for word in emoji_dict.values():
    word=re.sub(r":","", str(word))
    list_emoji_word.append(word)

#### 清除字典value中字串中所連接的_

In [73]:
list_emoji=[]
for word in list_emoji_word:
    if '_' in word:
        w = word.split("_")
        list_emoji.append(w)
    else:
        list_emoji.append([word])

#### 將字詞作詞形還原與關聯詞，計算情感分數

In [74]:
word_to_score = {}
lemmatizer = WordNetLemmatizer()

for i,sentence in enumerate(list_emoji):
    word_sentiment = 0.0
    for word in sentence:
        lemma = lemmatizer.lemmatize(word)
        if not lemma:
            continue
        
        synsets = wn.synsets(lemma)
        if not synsets:
            continue
        #字典字有限，要找和字典中的字最相關
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        word_sentiment += (swn_synset.pos_score() - swn_synset.neg_score())+1

    #以i取list_emoji_word的第i個數
    #key:emoji的描述，value：情感分數
    word_to_score[list_emoji_word[i]] = word_sentiment

#### 建立表情符號字典

In [75]:
emoji_score_dict={}
for key,value in emoji_dict.items():
    #把emoji_score_dict的value當word_to_score的key去抓分數
    emoji_score_dict[key] = word_to_score[value[1:-1]]

#### 計算評論表情符號比率&情感分數

In [76]:
emoji_list_2=[]

for sentence,count in zip(emot_list,remove_stw_count):
    
    emoji_count=0
    emoji_setiment_value=0.0
    
    #抓取文本中表情符號
    #value：字串中所有表情符號，location：表情符號在字串中的位置 ，mean：表情符號含意，flag：指定字串中是否存在表情符號

    #假設表情字串存在表情符號
    if len(sentence['value'])!=0:
        for word in sentence['value']:
            emoji_count+=1
            if  word in emoji_score_dict.keys():
                emoji_setiment_value+=emoji_score_dict[word]
                
    emoji_count=emoji_count/count
    
    emoji_temp_dict={'表情符號比率':emoji_count,'表情符號情感分數':emoji_setiment_value}
    df=pd.DataFrame([emoji_temp_dict])
    emoji_list_2.append(df)
emoji_data_2=pd.concat(emoji_list_2)  

In [77]:
emoji_data_2.to_csv (r"..\..\Feature_Array\PTT\Extra_Features\Sentiwordnet_Emoji_data.csv", header=True,encoding='utf_8_sig',index=None)

#### 情感特徵與表情符號合併

In [78]:
All_Sentiment_2=pd.concat([Sentiment_data.iloc[:, :],emoji_data_2], axis=1)

In [79]:
All_Sentiment_2.to_csv (r"..\..\Feature_Array\PTT\Extra_Features\All_Sentiment_Sentiwordnet.csv", header=True,encoding='utf_8_sig',index=None)